In [2]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [3]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('세조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세조실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세조실록"," "))
df=df.map(lambda x : x.replace("세조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

698

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.010551213286817074
	하니	0.009813112206757069
	관찰사	0.00862303376197815
	생각	0.007115360349416733
	모두	0.005591168999671936
	우리	0.005498225335031748
	어찌	0.005177500192075968
	다시	0.005110541358590126
	천순	0.004921694286167622
	군사	0.004901029635220766


Topic ID: 1
	백성	0.01638472080230713
	수령	0.010720900259912014
	모두	0.010587825439870358
	하니	0.010025080293416977
	어찌	0.006980813108384609
	반드시	0.006560693960636854
	자가	0.005533653777092695
	생각	0.005243103485554457
	만약	0.0047765388153493404
	가지	0.0047234175726771355


Topic ID: 2
	모두	0.011897957883775234
	백성	0.009089093655347824
	하니	0.008182834833860397
	어찌	0.0063730040565133095
	생각	0.005964262876659632
	수가	0.005561441648751497
	관찰사	0.005220034625381231
	군사	0.0052152955904603004
	우리	0.0047862334176898
	마음	0.004384665284305811


Topic ID: 3
	백성	0.015053895302116871
	군사	0.01167944073677063
	하니	0.010022162459790707
	모두	0.00837008561939001
	수령	0.0057884384877979755
	천순	0.005340941250324249
	어찌	0.00525577925145

In [12]:
# 백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.016034  0.013419       1        1  17.322264
12     0.013378 -0.015919       2        1  15.582599
1      0.025930 -0.032934       3        1  12.081695
3     -0.015778 -0.021207       4        1   7.193837
2      0.001477 -0.003051       5        1   5.297678
5     -0.003008  0.005129       6        1   5.143449
14     0.021857 -0.003772       7        1   4.212616
4      0.002902  0.023182       8        1   4.006569
15    -0.048459 -0.020437       9        1   3.430951
0     -0.002842 -0.002829      10        1   3.392367
19    -0.000301 -0.008139      11        1   3.176230
6      0.031904  0.009701      12        1   3.069449
10     0.038355  0.020373      13        1   3.037164
9     -0.009811  0.006096      14        1   2.342697
17    -0.006631 -0.004134      15        1   2.221826
7     -0.019041  0.019179      16        1   2.152487
13     0.002942 -0.045620      17        1   2.130916
8     -0.033618  0.020856      18        1   1.683992
16    -0.007690  0.018177      19        1   1.492658
11    -0.007602  0.021930      20        1   1.028557, topic_info=     Term         Freq        Total Category  logprob  loglift
20     백성  1351.000000  1351.000000  Default  30.0000  30.0000
72     모두   840.000000   840.000000  Default  29.0000  29.0000
323    군사   515.000000   515.000000  Default  28.0000  28.0000
49     하니   798.000000   798.000000  Default  27.0000  27.0000
441    우리   475.000000   475.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
1659   수가     2.874220   340.002899  Topic20  -5.8746  -0.1962
124    수령     2.827628   415.711884  Topic20  -5.8909  -0.4135
676    생각     2.959694   624.263733  Topic20  -5.8453  -0.7745
62     고을     2.242963   261.173828  Topic20  -6.1226  -0.1804
645    더욱     2.148940   219.287216  Topic20  -6.1654  -0.0484

[1585 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
10256      5  0.665765  가까스로
9285       8  0.418193    가난
2442       1  0.147129    가노
2442       2  0.147129    가노
2442       3  0.147129    가노
...      ...       ...   ...
2092      17  0.189942    흐름
8075       3  0.650433    흥이
296        1  0.327905    흥평
296       12  0.327905    흥평
7109       1  0.642343    희문

[4807 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 13, 2, 4, 3, 6, 15, 5, 16, 1, 20, 7, 11, 10, 18, 8, 14, 9, 17, 12])

In [13]:
# 군사

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('세조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '군사' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세조실록'))

In [15]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세조실록"," "))
df=df.map(lambda x : x.replace("세조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [16]:
len(df)

1127

In [17]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [18]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [19]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [20]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	군사	0.02155858464539051
	하니	0.0080484077334404
	하소	0.007748947013169527
	천순	0.007008586078882217
	다시	0.005402524024248123
	모두	0.004807945340871811
	어찌	0.004667160101234913
	청컨대	0.004631604067981243
	생각	0.004382146522402763
	우리	0.00409846194088459


Topic ID: 1
	군사	0.013825283385813236
	모두	0.00950364675372839
	천순	0.0065256268717348576
	하니	0.006363277789205313
	병조	0.006135917268693447
	판서	0.005212032701820135
	하소	0.00488313427194953
	우리	0.0047986069694161415
	관찰사	0.003768037771806121
	제사	0.0035163802094757557


Topic ID: 2
	군사	0.012196699157357216
	하니	0.010171259753406048
	모두	0.00585537264123559
	우리	0.005360190290957689
	각각	0.005182311870157719
	마음	0.004911512136459351
	반드시	0.004521001596003771
	어찌	0.004499533679336309
	자가	0.0043250867165625095
	천순	0.004322102293372154


Topic ID: 3
	군사	0.013994095847010612
	모두	0.005539417266845703
	천순	0.005519254598766565
	백성	0.005045099649578333
	생각	0.004965519066900015
	우리	0.004961684811860323
	가지	0.0047500883229076

In [21]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.016891  0.046405       1        1  14.420408
10     0.031819 -0.034764       2        1  10.772157
9     -0.015300  0.015723       3        1  10.558917
16     0.038774  0.016988       4        1   8.626732
2      0.016843 -0.007425       5        1   6.739393
1     -0.008645  0.005380       6        1   5.243605
4      0.012123  0.005207       7        1   4.866026
17     0.002276  0.000838       8        1   4.376697
19     0.020561  0.013035       9        1   4.028088
5      0.008072  0.024244      10        1   3.789239
18    -0.078005  0.018292      11        1   3.723477
7     -0.002137 -0.006526      12        1   3.357197
13     0.008899  0.012438      13        1   3.135304
14     0.005149 -0.010967      14        1   2.969067
0      0.001089 -0.012944      15        1   2.950387
6     -0.010324  0.003707      16        1   2.777457
3     -0.003777 -0.038554      17        1   2.153660
11     0.003687 -0.008008      18        1   1.967805
15    -0.034283 -0.012224      19        1   1.775501
8     -0.013712 -0.030845      20        1   1.768884, topic_info=     Term         Freq        Total Category  logprob  loglift
7      군사  2410.000000  2410.000000  Default  30.0000  30.0000
125    모두   941.000000   941.000000  Default  29.0000  29.0000
609    하소   466.000000   466.000000  Default  28.0000  28.0000
40     천순   591.000000   591.000000  Default  27.0000  27.0000
326    병조   443.000000   443.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
45     하니     8.794944  1025.514771  Topic20  -5.5551  -0.7240
4232   성화     6.432223   412.324738  Topic20  -5.8680  -0.1257
53     가지     6.535572   464.892456  Topic20  -5.8520  -0.2297
40     천순     6.584110   591.540833  Topic20  -5.8446  -0.4633
609    하소     6.044652   466.821350  Topic20  -5.9301  -0.3120

[1605 rows x 6 columns], token_table=       Topic      Freq Term
term                       
11676     12  0.672446  가마니
12962     13  0.530738   가전
53         1  0.116156   가지
53         2  0.101099   가지
53         3  0.129062   가지
...      ...       ...  ...
6566      12  0.295107   훈척
7748       6  0.354156   흥녕
7748      13  0.354156   흥녕
2707       5  0.515674  흥부가
7387       2  0.818245   희전

[5570 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 11, 10, 17, 3, 2, 5, 18, 20, 6, 19, 8, 14, 15, 1, 7, 4, 12, 16, 9])

In [ ]:
# 심신

In [22]:
data =[]
for i in range(0,1):
    txt1 = open('세조실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세조실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '심신' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세조실록'))

In [23]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세조실록"," "))
df=df.map(lambda x : x.replace("세조"," "))
df=df.map(lambda x : x.replace("번째기사"," "))
len(df)

12

In [24]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [25]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	유계	0.019679652526974678
	정언	0.019568590447306633
	종손	0.019201893359422684
	심신	0.01622161827981472
	아내	0.01457833219319582
	경태	0.011498929001390934
	중추원	0.010871974751353264
	병조	0.010535321198403835
	신민	0.010147231630980968
	병인	0.009739023633301258


Topic ID: 1
	하니	0.011173181235790253
	이계전	0.0106123685836792
	이개	0.010222441516816616
	정효상	0.010067368857562542
	심신	0.009735642932355404
	박팽년	0.009533816948533058
	어찌	0.00913373101502657
	김인	0.008847539313137531
	참여	0.008707992732524872
	역모	0.008025355637073517


Topic ID: 2
	아비	0.008508243598043919
	하니	0.007856223732233047
	어찌	0.007778992410749197
	전지	0.006745212711393833
	의문	0.006618890445679426
	옥산	0.00622197100892663
	이개	0.005475401412695646
	사살	0.005195151083171368
	만약	0.0046571651473641396
	박팽년	0.004622494801878929


Topic ID: 3
	권경	0.0010683761211112142
	군수	0.0010683761211112142
	권언	0.0010683761211112142
	권반	0.0010683761211112142
	권남	0.0010683761211112142
	권공	0.0010683761211112142
	귀비	0.0010683761

In [26]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.164908  0.063013       1        1  25.156164
11     0.047698  0.068035       2        1  19.813818
4     -0.278865 -0.022774       3        1  19.657103
9      0.027287 -0.053864       4        1  12.043438
12     0.037372  0.033757       5        1  11.588634
16     0.052308 -0.102308       6        1   5.145936
17    -0.040232 -0.106676       7        1   4.498837
19     0.002286  0.092303       8        1   0.907049
0      0.038306  0.032990       9        1   0.858358
2      0.044734 -0.022525      10        1   0.036862
10    -0.004311  0.003439      11        1   0.033705
8      0.039568 -0.039444      12        1   0.032670
15    -0.000490  0.040726      13        1   0.031603
13     0.082793  0.037778      14        1   0.031490
1      0.053626 -0.026749      15        1   0.031428
7      0.052642  0.031687      16        1   0.028464
5      0.003842  0.022108      17        1   0.028372
6      0.013677 -0.024076      18        1   0.028094
14    -0.007104 -0.009936      19        1   0.025318
3     -0.000228 -0.017484      20        1   0.022659, topic_info=    Term        Freq       Total Category  logprob  loglift
453   아내   87.000000   87.000000  Default  30.0000  30.0000
546   전지  111.000000  111.000000  Default  29.0000  29.0000
127   하니   12.000000   12.000000  Default  28.0000  28.0000
27    대답    8.000000    8.000000  Default  27.0000  27.0000
71    심신   12.000000   12.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
296   박대    0.000471    3.159682  Topic20  -6.8416  -0.4186
295   바람    0.000471    1.121343  Topic20  -6.8416   0.6174
294   문득    0.000471    1.063901  Topic20  -6.8416   0.6700
320   성대    0.000471    1.082235  Topic20  -6.8416   0.6529
321   성승    0.000471    9.167852  Topic20  -6.8416  -1.4838

[1144 rows x 6 columns], token_table=      Topic      Freq Term
term                      
135       4  0.687306   가루
135       7  0.687306   가루
1         2  0.740224  가운데
2         1  0.214304   가지
2         2  0.214304   가지
...     ...       ...  ...
185       7  0.378392   효과
830       1  0.416667   효도
935       3  0.626047   효령
832       1  0.428939   효생
835       1  0.776618   효의

[584 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 12, 5, 10, 13, 17, 18, 20, 1, 3, 11, 9, 16, 14, 2, 8, 6, 7, 15, 4])